In [1]:
!pip install awswrangler
%additional_python_modules awswrangler

import boto3
import datetime
import awswrangler as wr
from pyspark.sql import DataFrame
from pyspark.sql.session import SparkSession
from pyspark.context import SparkContext
from pyspark.sql.utils import AnalysisException
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext

session = boto3.session.Session(region_name='eu-west-1')
s3 = session.client('s3')
glue = session.client('glue')
spark = SparkSession.builder.config('spark.serializer','org.apache.spark.serializer.KryoSerializer').config('spark.sql.hive.convertMetastoreParquet','false').getOrCreate()
sc = spark.sparkContext
glueContext = GlueContext(SparkContext.getOrCreate())


Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Additional python modules to be included:
awswrangler



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Trying to create a Glue session for the kernel.
Session Type: etl
Session ID: 9f172e7d-da0d-43b7-9b21-a834ad7e64d7
Applying the following default arguments:
--glue_kernel_version 1.0.6
--enable-glue-datacatalog true
--additional-python-modules awswrangler
Waiting for session 9f172e7d-da0d-43b7-9b21-a834ad7e64d7 to get into ready status...
Session 9f172e7d-da0d-43b7-9b21-a834ad7e64d7 has been created.



In [3]:
env        = 'dev'
output_full_path = 's3://sds-dev-transform-dwh-etl/ym_fct_competitor_prices_14_day_como_full/'
output_full_path_tmp = 's3://sds-dev-transform-dwh-etl/ym_fct_competitor_prices_14_day_tmp_como_full/'
output_limited_path = 's3://sds-dev-transform-dwh-etl/ym_fct_competitor_prices_14_day_como_limited/'
output_limited_path_tmp = 's3://sds-dev-transform-dwh-etl/ym_fct_competitor_prices_14_day_tmp_como_limited/'

In [12]:

SqlQuery = """
WITH base as (
 SELECT
    rank() OVER (PARTITION BY protection, competitor, CASE WHEN substr(shop_detail,6,3) = 'GDS' THEN 'GDS' ELSE 'No GDS' END, broker, mapped_scrs, mapped_fmod, mapped_crs, length_of_rent, pickup_date_time, pickup_station, return_date_time, return_station, tariff_type, payment_type, customer_no, mapped_mileage, point_of_sales, original_vehicle_category, original_crs, original_mileage ORDER BY created_at DESC, shop_date_time DESC, (CASE WHEN (COALESCE(converted_price, 0) = 0) THEN original_price ELSE converted_price END) ASC, rand() ASC) row_num
   , competitor
   , broker
   , shop_date_time
   , shop_detail
   , CASE WHEN substr(shop_detail,6,3) = 'GDS' THEN 'GDS' ELSE 'No GDS' END AS GDS
   , converted_price
   , original_price
   , converted_currency
   , original_currency
   , original_vehicle_category
   , original_crs
   , original_sample_vehicle
   , mapped_scrs
   , mapped_fmod
   , mapped_crs
   , length_of_rent
   , pickup_date_time
   , pickup_station
   , return_date_time
   , return_station
   , tariff_type
   , payment_type
   , customer_no
   , original_mileage
   , mapped_mileage
   , point_of_sales
   , rate_level
   , sixt_plan
   , protection
   FROM competitorprice)
SELECT base.protection AS YCPR_PROTECTION_FLG_STRING
    , (CASE WHEN (base.competitor <> 'Sixt') THEN 'Competitors' WHEN (CAST(base.protection AS integer) = 1) THEN 'Inclusive' ELSE 'Exclusive' END) AS YCPR_PROTECTION_SIXT
    , (CASE WHEN (base.competitor = 'Sixt') THEN 'Sixt' WHEN (CAST(base.protection AS integer) = 1) THEN 'Inclusive' ELSE 'Exclusive' END) AS YCPR_PROTECTION_COMPETITORS
    , base.competitor AS YCPR_COMPETITOR
    , base.broker AS YCPR_BROKER
    , CAST(base.shop_date_time AS timestamp) AS YCPR_SHOP_DATM
    , concat(date_format(CAST(base.shop_date_time AS timestamp), 'y'), '_', extract(week FROM shop_date_time)) AS YCPR_SHOP_WEEK
    , CAST(date_format(CAST(base.shop_date_time AS timestamp), 'yMMdd') AS integer) YCPR_SHOP_DTID
    , datediff(current_date, date(CAST(base.shop_date_time AS timestamp))) YCPR_DATE_DIFF
    , base.shop_detail YCPR_SHOP_DETAIL
    , base.GDS YCPR_GDS
    , CAST((CASE WHEN (COALESCE(converted_price, 0) = 0) THEN original_price ELSE converted_price END) AS decimal(10,2)) YCPR_FINAL_PRICE
    , (CASE WHEN (base.converted_currency IS NULL) THEN base.original_currency ELSE CAST(converted_currency AS string) END) YCPR_FINAL_CURRENCY
    , base.original_vehicle_category YCPR_VEHICLE_CATEGORY_ORIGINAL
    , base.original_crs YCPR_CRS_ORIGINAL
    , base.original_sample_vehicle YCPR_SAMPLE_VEHICLE_ORIGINAL
    , base.mapped_scrs YCPR_SCRS_MAPPED
    , base.mapped_fmod YCPR_FMOD_MAPPED
    , base.mapped_crs YCPR_CRS_MAPPED
    , (CASE WHEN base.mapped_crs like '%E' THEN 'E'
                WHEN substr(base.mapped_crs,4,1) = 'H' THEN 'H'
                WHEN base.mapped_crs in ('CCCC', 'FCCC', 'PCCC', 'CDHC', 'SFAC', 'SPAC', 'LFAC', 'XFAC') THEN 'O'
                WHEN substr(base.mapped_crs,4,1) = 'C' THEN 'E'
                ELSE 'O'
            END) AS VHCL_ELECTRIC
   
    , CASE WHEN vg.vhgr_type_code in ('P', 'B', 'M')  then 'C&B' when (vg.vhgr_type_code='T' and base.mapped_crs<>'P') then 'C&B' else 'V&T' end as YCPR_VEHICLE_TYPE    
    , CAST(base.length_of_rent AS integer) YCPR_RENTAL_DAYS
    , CAST(base.length_of_rent AS string) YCPR_RENTAL_DAYS_STRING
    , CAST(base.pickup_date_time AS timestamp) YCPR_HANDOVER_DATM
    , CAST(date(CAST(base.pickup_date_time AS timestamp)) AS string) YCPR_HANDOVER_DATE
    , date_trunc('week', CAST(base.pickup_date_time AS timestamp)) YCPR_HANDOVER_WEEK_DATE
    , concat(date_format(CAST(base.pickup_date_time AS timestamp), 'y'), '_', extract(week FROM base.pickup_date_time)) YCPR_HANDOVER_WEEK
    , date_format(CAST(base.pickup_date_time AS timestamp), 'y_MM') YCPR_HANDOVER_MONTH
    , date_format(CAST(base.pickup_date_time AS timestamp), 'EEEE') YCPR_HANDOVER_DAY
    , CAST(date_format(CAST(base.pickup_date_time AS timestamp), 'yMMdd') AS integer) YCPR_HANDOVER_DTID
    , CAST(base.return_date_time AS timestamp) YCPR_RETURN_DATM
    , CAST(date_format(CAST(base.return_date_time AS timestamp), 'yMMdd') AS integer) YCPR_RETURN_DTID
    , CAST(base.pickup_station AS integer) BRNC_HANDOVER_CODE
    , concat(CAST(br.brnc_code AS string), ' ', br.brnc_name) BRNC_HANDOVER_NAME
    , br.brnc_country_code_iso BRNC_HANDOVER_COUNTRY
    , br.brnc_region_code BRNC_HANDOVER_REGION_CODE
    , concat(concat(CAST(br.brnc_region_code AS string), ' '), br.brnc_region) BRNC_HANDOVER_REGION
    , br.brnc_pool_code BRNC_HANDOVER_POOL_CODE
    , concat(concat(CAST(br.brnc_pool_code AS string), ' '), br.brnc_pool_name) BRNC_HANDOVER_POOL
    , CAST(base.return_station AS string) BRNC_RETURN_CODE
    , (CASE WHEN (base.pickup_station = CAST(base.return_station AS integer)) THEN 'N' ELSE 'Y' END) BRNC_ONE_WAY
    , base.tariff_type YCPR_TARIFF_TYPE
    , base.payment_type YCPR_PAYMENT_TYPE
    , base.customer_no YCPR_CUSTOMER_NUMBER
    , CAST(base.original_mileage AS bigint) YCPR_MILEAGE_ORIGINAL
    , CAST(base.mapped_mileage AS string) YCPR_MILEAGE_MAPPED
    , base.point_of_sales YCPR_POSL_COUNTRY_CODE
    , CASE WHEN (br.brnc_country_code_iso = 'MC' and base.point_of_sales='FR') THEN 'Domestic'
        WHEN base.point_of_sales=br.brnc_country_code_iso THEN 'Domestic'
        else 'Inbound' end as YCPR_DOMESTIC_INBOUND
    , base.rate_level YCPR_RATE_LEVEL
    , cast(base.sixt_plan as string) YCPR_SIXT_PLAN_STRING
    , base.row_num YCPR_RECORD_SEQUENCE
    , current_timestamp() as YCPR_CURRENT_TIMESTAMP
FROM base
LEFT JOIN branches br ON ((CAST(base.pickup_station AS integer) = br.brnc_code) AND (br.sys_deleted_flg = 0))
LEFT JOIN vehiclegroups vg ON ((base.mapped_crs = vg.vhgr_crs) AND (vg.sys_deleted_flg = 0))
WHERE (row_num IN (1)) AND ( case when br.brnc_country_code_iso in ('US', 'CA') then CAST(base.pickup_date_time AS timestamp) >= date_add(current_date, -1) else CAST(base.pickup_date_time AS timestamp) >= current_date end )
"""


You are already connected to a glueetl session c87d800c-6c31-404d-bf86-1d665b4f545a.

No change will be made to the current session that is set as glueetl. The session configuration change will apply to newly created sessions.


Setting Glue version to: 4.0


In [9]:
t = datetime.datetime.now()
y_m_d_folder = []
for i in range(16):
    date_to_check = t - datetime.timedelta(days=i)
    year=date_to_check.year
    day_0 = str(date_to_check.day).zfill(2)
    month_0 = str(date_to_check.month).zfill(2)
    #print(year, month_0, day_0)

    folder_path = f's3://sds-{env}-ingest-external-sftp-files-out/yield-sftp-user/CompetitorPrice/year={year}/month={month_0}/day={day_0}/'
    #print(folder_path)   
    y_m_d_folder.append(folder_path)
    
print(y_m_d_folder)    


['s3://sds-dev-ingest-external-sftp-files-out/yield-sftp-user/CompetitorPrice/year=2024/month=08/day=27/', 's3://sds-dev-ingest-external-sftp-files-out/yield-sftp-user/CompetitorPrice/year=2024/month=08/day=26/', 's3://sds-dev-ingest-external-sftp-files-out/yield-sftp-user/CompetitorPrice/year=2024/month=08/day=25/', 's3://sds-dev-ingest-external-sftp-files-out/yield-sftp-user/CompetitorPrice/year=2024/month=08/day=24/', 's3://sds-dev-ingest-external-sftp-files-out/yield-sftp-user/CompetitorPrice/year=2024/month=08/day=23/', 's3://sds-dev-ingest-external-sftp-files-out/yield-sftp-user/CompetitorPrice/year=2024/month=08/day=22/', 's3://sds-dev-ingest-external-sftp-files-out/yield-sftp-user/CompetitorPrice/year=2024/month=08/day=21/', 's3://sds-dev-ingest-external-sftp-files-out/yield-sftp-user/CompetitorPrice/year=2024/month=08/day=20/', 's3://sds-dev-ingest-external-sftp-files-out/yield-sftp-user/CompetitorPrice/year=2024/month=08/day=19/', 's3://sds-dev-ingest-external-sftp-files-out/

In [ ]:
current_date = datetime.datetime.now()
folder_template = 's3://sds-{}-ingest-external-sftp-files-out/yield-sftp-user/CompetitorPrice/year={}/month={:02}/day={:02}/'
source_folders = [folder_template.format(env, d.year, d.month, d.day) 
                  for d in [current_date - datetime.timedelta(days=i) for i in range(16)]]
print(source_folders)

In [45]:
def path_exists(path: str) -> bool:
    files = wr.s3.list_objects(path=path, s3_additional_kwargs={'MaxKeys': 1})
    return len(files) > 0
existing_folders = [f for f in source_folders if path_exists(f)]

df = spark.read.parquet(*existing_folders)
df.createOrReplaceTempView('source_data')

source_competitor_price = spark.sql("""
        SELECT * 
        FROM source_data
        WHERE (pickup_date_time >= date_sub(current_date, 1))
    """)
    
source_competitor_price.show()    

+--------------------+-------------------+--------------------+---------------+------------------+--------------+-----------------+-------------------------+------------+-----------------------+------------+-------------+----------+--------------+--------------------+--------------+--------------------+--------------+----------+-----------+----------+---------------+-------+-----------+----------------+--------------+--------------+----------+---------+------------+--------------------+----+----+----+-------------+--------------+
|      shop_date_time|        shop_source|         shop_detail|converted_price|converted_currency|original_price|original_currency|original_vehicle_category|original_crs|original_sample_vehicle| mapped_scrs|  mapped_fmod|mapped_crs|length_of_rent|    pickup_date_time|pickup_station|    return_date_time|return_station|protection|tariff_type|competitor|         source| broker|customer_no|original_mileage|mapped_mileage|point_of_sales|rate_level|sixt_plan|payment